### MANDATORY ASSIGNMENT 2

In [2]:
from sklearn import datasets

In [3]:
import numpy as np

In [4]:
iris = datasets.load_iris()

In [5]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [6]:
len(X)

150

In [7]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [8]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


In [11]:
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import SPSA
import random
from sklearn.metrics import log_loss # loss function
from sklearn.metrics import accuracy_score # accuracy

In [10]:
scaler = MinMaxScaler(feature_range=(0, np.pi)) # since we are using angle encoding, we need to scale from 0 to pi
X = scaler.fit_transform(X)

In [83]:
class QuantumMachineLearning:
    def __init__(self, X_train, y_train, X_val, y_val, num_qubits = 4, num_layers = 3):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = len(set(y_train))
        self.rng = np.random.default_rng(42)
        self.initial_parameters = self.rng.uniform(0, np.pi, self.num_qubits * self.num_layers)
    
    def angle_encoding(self, qc, sample):
        for qubit in range(len(qc.qubits)):
            qc.rx(sample[qubit], qubit)
    

    def real_amplitudes(self, data_point, parameters):
        qc = QuantumCircuit(self.num_qubits)
        self.angle_encoding(qc, data_point)

        param_index = 0

        for layer in range(self.num_layers):
            for qubit in range(len(qc.qubits)):
                qc.ry(parameters[param_index], qubit)
                param_index += 1
            qc.barrier()
            
            for qubit in range(len(qc.qubits)-1):
                qc.cx(qubit, qubit+1)
            qc.barrier()

        return qc
    
    def run_circuit(self, data_point, params, circuit = 'real_amplitudes', shots = 100):
        backend = AerSimulator(method = 'statevector')
        if circuit == 'real_amplitudes':
            qc = self.real_amplitudes(data_point, params)
            
        qc.measure_all()

        tqc = transpile(qc, backend)

        job = backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)

        return counts

    
    def data_decoding(self, output):
        return int(output, 2) % self.num_classes
    
    def loss_function(self, updated_params):
        shots = 100
        predicted_probabilites = []
        for x in self.X_train:
            counts = self.run_circuit(x, updated_params, shots = shots)

            count_classes = {x : 0 for x in range(self.num_classes)}
            for output, count in counts.items():
                class_num = self.data_decoding(output)
                count_classes[class_num] += count / shots
            

            predicted_probabilites.append([count_classes[x] for x in range(self.num_classes)])        
            
        
        logloss = log_loss(self.y_train, predicted_probabilites)

        #print(f"Parameters: {updated_params} loss: {logloss}")
        return logloss

    def SPSA_optimize(self, maxiter = 50):
        optimizer = SPSA(maxiter=maxiter)
        # Optimize the parameters
        optimized = optimizer.minimize(fun=self.loss_function, x0=self.initial_parameters)

        print("Optimized Parameters:", optimized.x)
        print("Minimum Loss:", optimized.fun)
        self.optimized_params = optimized.x
        self.min_loss = optimized.fun
    
    def save_parameters(self):
        with open('optimized_parameters.txt', 'w') as file:
            file.write(str(self.optimized_params))
    

    def gradient(self, params, epsilon = 0.2):
        gradients = []

        for i in range(len(params)):
            plus = [params[j] + epsilon if j == i else params[j] for j in range(len(params))]
            minus = [params[j] - epsilon if j == i else params[j] for j in range(len(params))]

            gradient = (self.loss_function(plus) - self.loss_function(minus)) / (2 * epsilon)
            gradients.append(gradient)

        return gradients
    
    
    def gradient_descent(self, learning_rate = 0.1, maxiter = 50):
        current_point = self.initial_parameters

        for i in range(maxiter):
            gradients = self.gradient(current_point)
            current_point = [current_point[j] - learning_rate * gradients[j] for j in range(len(gradients))]
            loss = self.loss_function(current_point)
            print(f"Iteration {i} Loss: {loss}")
        
        print("Optimized Parameters:", current_point, "Loss:", loss)
        self.optimized_params = current_point


    
    
    def predict(self, data_point): #paramteres must be optimized before prediction
        prediction_shots = 100000 # more shots as the circuit is only run once
        counts = self.run_circuit(data_point, self.optimized_params, shots = prediction_shots)

        predicted_probabilites = {x : 0 for x in range(self.num_classes)}
            
        # Decode each measurement outcome and aggregate probabilites for each class
        for output, count in counts.items():
            class_num = self.data_decoding(output)
            predicted_probabilites[class_num] += count / prediction_shots
        
        
        # Determine the predicted class by choosing the class with the highest probability
        predicted_class = max(predicted_probabilites, key=predicted_probabilites.get)
        
        return predicted_class

    def predict_dataset(self, X):
        return [self.predict(data_point) for data_point in X]
    
    def performance(self, y_test, X_test):
        return accuracy_score(y_test, self.predict_dataset(X_test))
    
    
    

In [84]:
from sklearn.model_selection import train_test_split

In [79]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [75]:
spsa = QuantumMachineLearning(X_train, y_train)
spsa.SPSA_optimize()

Parameters: [2.63145464 1.17877728 2.89736492 2.39084628 0.49586686 2.86500801
 2.5911909  2.26949385 0.20248085 1.61492915 0.96489635 2.71151808] loss: 0.998770732615339
Parameters: [2.23145464 1.57877728 2.49736492 1.99084628 0.09586686 3.26500801
 2.1911909  2.66949385 0.60248085 1.21492915 1.36489635 3.11151808] loss: 1.2723476736941475
Parameters: [2.23145464 1.17877728 2.49736492 2.39084628 0.49586686 2.86500801
 2.1911909  2.66949385 0.20248085 1.21492915 1.36489635 3.11151808] loss: 1.1251839431082862
Parameters: [2.63145464 1.57877728 2.89736492 1.99084628 0.09586686 3.26500801
 2.5911909  2.26949385 0.60248085 1.61492915 0.96489635 2.71151808] loss: 0.9895173470599065
Parameters: [2.23145464 1.57877728 2.49736492 2.39084628 0.09586686 2.86500801
 2.5911909  2.26949385 0.60248085 1.21492915 1.36489635 3.11151808] loss: 1.1037163613558378
Parameters: [2.63145464 1.17877728 2.89736492 1.99084628 0.49586686 3.26500801
 2.1911909  2.66949385 0.20248085 1.61492915 0.96489635 2.7115

In [77]:
spsa.save_parameters()
spsa.performance(y_test, X_test)

0.9130434782608695

In [88]:
ours = QuantumMachineLearning(X_train, y_train)
ours.gradient_descent(learning_rate=0.5, maxiter=100)

Iteration 0 Loss: 1.0445334984632195
Iteration 1 Loss: 0.9925891454689159
Iteration 2 Loss: 0.945551791906504
Iteration 3 Loss: 0.9323700530917735
Iteration 4 Loss: 0.9023047300957192
Iteration 5 Loss: 0.8846784378897742
Iteration 6 Loss: 0.8614149246932473
Iteration 7 Loss: 0.8483816342153252
Iteration 8 Loss: 0.8321334545476545
Iteration 9 Loss: 0.8282281750553359
Iteration 10 Loss: 0.8132293218687842
Iteration 11 Loss: 0.8111309373200807
Iteration 12 Loss: 0.7930501685926176
Iteration 13 Loss: 0.7911531626226448
Iteration 14 Loss: 0.7845376799414653
Iteration 15 Loss: 0.7691073324008648
Iteration 16 Loss: 0.7663872384979388
Iteration 17 Loss: 0.7641348824037416
Iteration 18 Loss: 0.7532199301132393
Iteration 19 Loss: 0.7575587348507932
Iteration 20 Loss: 0.746092949110698
Iteration 21 Loss: 0.7450456371640717
Iteration 22 Loss: 0.743193398679631
Iteration 23 Loss: 0.7408188844649771
Iteration 24 Loss: 0.7398960354584693
Iteration 25 Loss: 0.7417773557386574
Iteration 26 Loss: 0.7329

In [96]:
ours.save_parameters()
ours.performance(y_test, X_test)

0.8260869565217391